In [ ]:
# Import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import seaborn as sns
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, balanced_accuracy_score, brier_score_loss
from sklearn.isotonic import IsotonicRegression
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.preprocessing.image import ImageDataGenerator

sns.set()

In [ ]:


# 1. Exploration
# Read csv file containing training data
train_df = pd.read_csv("data/nih/train-small.csv")

# Print first 5 rows
print(f'There are {train_df.shape[0]} rows and {train_df.shape[1]} columns in this data frame')
train_df.head()

# 2. Image preprocessing
# Define image size and batch size
IMG_SIZE = 224
BATCH_SIZE = 32

# Data generator with augmentation for training data
datagen_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data generator for validation data
datagen_val = ImageDataGenerator(rescale=1./255)

# 3. Patient overlap and data leakage
# Assuming train_df contains columns 'PatientID' and 'ImagePath'
train_df['PatientID'] = train_df['ImagePath'].apply(lambda x: x.split('_')[0])
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['PatientID'])

# Ensure no patient overlap between training and validation sets
print(f'Number of unique patients in training set: {train_df["PatientID"].nunique()}')
print(f'Number of unique patients in validation set: {val_df["PatientID"].nunique()}')
print(f'Patient overlap: {set(train_df["PatientID"]).intersection(set(val_df["PatientID"]))}')

# 4. Training pretrained CNN
# Load the DenseNet121 model pre-trained on ImageNet
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Data generators
train_generator = datagen_train.flow_from_dataframe(
    dataframe=train_df,
    x_col='ImagePath',
    y_col='FindingLabel',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

val_generator = datagen_val.flow_from_dataframe(
    dataframe=val_df,
    x_col='ImagePath',
    y_col='FindingLabel',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

# Train the model
model.fit(train_generator, validation_data=val_generator, epochs=10)

# 5. Generate prediction metrics (AUC, balanced accuracy, brier score, ECE and plot curve)
# Predict on validation set
val_generator.reset()
preds = model.predict(val_generator)
y_true = val_df['FindingLabel'].values

# Calculate AUC and balanced accuracy
auc = roc_auc_score(y_true, preds)
balanced_acc = balanced_accuracy_score(y_true, preds.round())

# Calculate Brier score
brier = brier_score_loss(y_true, preds)

# Calculate ECE
prob_true, prob_pred = calibration_curve(y_true, preds, n_bins=10)
ece = np.mean(np.abs(prob_true - prob_pred))

# Plot calibration curve
plt.figure(figsize=(10, 10))
plt.plot(prob_pred, prob_true, marker='o', linewidth=1, label='Model')
plt.plot([0, 1], [0, 1], linestyle='--', label='Perfectly calibrated')
plt.xlabel('Mean predicted probability')
plt.ylabel('Fraction of positives')
plt.title('Calibration plot')
plt.legend()
plt.show()

print(f'AUC: {auc}')
print(f'Balanced Accuracy: {balanced_acc}')
print(f'Brier Score: {brier}')
print(f'Expected Calibration Error (ECE): {ece}')

# 6. Enhance model calibration with Platt Scaling
# Fit logistic regression for Platt Scaling
platt_model = LogisticRegression()
platt_model.fit(preds.reshape(-1, 1), y_true)

# Transform predictions
platt_preds = platt_model.predict_proba(preds.reshape(-1, 1))[:, 1]

# 7. Generate again prediction metrics
# Calculate new metrics after Platt Scaling
auc_platt = roc_auc_score(y_true, platt_preds)
balanced_acc_platt = balanced_accuracy_score(y_true, platt_preds.round())
brier_platt = brier_score_loss(y_true, platt_preds)
prob_true_platt, prob_pred_platt = calibration_curve(y_true, platt_preds, n_bins=10)
ece_platt = np.mean(np.abs(prob_true_platt - prob_pred_platt))

# Plot calibration curve after Platt Scaling
plt.figure(figsize=(10, 10))
plt.plot(prob_pred_platt, prob_true_platt, marker='o', linewidth=1, label='Platt Scaled Model')
plt.plot([0, 1], [0, 1], linestyle='--', label='Perfectly calibrated')
plt.xlabel('Mean predicted probability')
plt.ylabel('Fraction of positives')
plt.title('Calibration plot after Platt Scaling')
plt.legend()
plt.show()

print(f'AUC after Platt Scaling: {auc_platt}')
print(f'Balanced Accuracy after Platt Scaling: {balanced_acc_platt}')
print(f'Brier Score after Platt Scaling: {brier_platt}')
print(f'Expected Calibration Error (ECE) after Platt Scaling: {ece_platt}')
